In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.model_selection import train_test_split
import math as mt
from math import e
from sklearn.preprocessing import StandardScaler

In [2]:
# Read Orography
OroData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')

In [3]:
#Surface data
t2mData = xr.open_dataset('../../../Data/CMIP6/INM-CM4-8/processed/Regrid_tas_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
rhData  = xr.open_dataset('../../../Data/CMIP6/INM-CM4-8/processed/Regrid_hurs_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
u10Data = xr.open_dataset('../../../Data/CMIP6/INM-CM4-8/processed/Regrid_uas_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
v10Data = xr.open_dataset('../../../Data/CMIP6/INM-CM4-8/processed/Regrid_vas_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')

In [4]:
# Level Data
tLevData = xr.open_dataset('../../../Data/CMIP6/INM-CM4-8/processed/Regrid_ta_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
zLevData = xr.open_dataset('../../../Data/CMIP6/INM-CM4-8/processed/Regrid_zg_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
wLevData = xr.open_dataset('../../../Data/CMIP6/INM-CM4-8/processed/Regrid_wap_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')
uLevData = xr.open_dataset('../../../Data/CMIP6/INM-CM4-8/processed/Regrid_ua_day_INM-CM4-8_historical_r1i1p1f1_gr1_18500101-20141231.nc')

In [5]:
t2mD = t2mData.sel(latitude=slice(32,24),longitude=slice(74,86))
u10D = u10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
v10D = v10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
rhD  = rhData.sel(latitude=slice(32,24),longitude=slice(74,86))
tLevD = tLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
uLevD = uLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
w700D = wLevData.sel(latitude=slice(32,24),longitude=slice(74,86)).sel(plev=70000,method='nearest')
zLevD = zLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
OroD = OroData.sel(latitude=slice(32,24),longitude=slice(74,86))
#w700D

In [6]:
# Calculate wind speed and relative humidity inv  ushear
ws = ((v10D.vas.values**2)+(u10D.uas.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhD.hurs)},
                   coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

#Calculate inv
inv=t2mD.tas.values-tLevD.ta.sel(plev=85000,method='nearest').values
inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, 
                    coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
inv_ds.attrs
inv_ds.attrs['units']='K'
inv_ds.attrs['long_name']='t2m - t850'

#u shear calculation
ushear=(uLevD.ua.sel(plev=85000,method='nearest').values-u10D.uas.values)/(zLevD.zg.sel(plev=85000,method='nearest').values) 
ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, 
                       coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
ushear_ds.attrs['units']='s-1'
ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [7]:
AOData = xr.open_dataset('INM-CM4-8-AOindex-NDJF-Daily-1980-2014.nc')

AO=AOData.AO
datetimeindex = AO.indexes['time'].to_datetimeindex()#
datetimeindex
AO['time'] = datetimeindex
AO.values

AO5D=AO.rolling(time=5).mean()

AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
#ao

<ipython-input-7-6f239f3b4c90>:4: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = AO.indexes['time'].to_datetimeindex()#


In [8]:
EUData = xr.open_dataset('INM-CM4-8-EUindex-NDJF-Daily-1980-2014.nc')
EU=EUData.EU
datetimeindex = EU.indexes['time'].to_datetimeindex()
datetimeindex
EU['time'] = datetimeindex
EU.values
EU5D=EU.rolling(time=5).mean()

EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))


<ipython-input-8-05bba2f93b3e>:3: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = EU.indexes['time'].to_datetimeindex()


In [9]:
# create mask
oro = OroD.z
oro.values = OroD.z.values/9.81
oro.attrs
oro.attrs['units']='meter'
oro.attrs['long_name']='Orography'
oro.values[oro.values>500.1]=np.NaN
mask=oro.values/oro.values

In [10]:
# Apply Mask 
t2mD.tas.values=t2mD.tas.values*mask
ws_ds.ws.values=ws_ds.ws.values*mask
rh_ds.rh.values=rh_ds.rh.values*mask
w700D.wap.values=w700D.wap.values*mask
inv_ds.inv.values=inv_ds.inv.values*mask
ushear_ds.ushear.values=ushear_ds.ushear.values*mask

In [11]:
# Feild mean
t2mTS1=t2mD.mean(dim=['latitude','longitude'], skipna=True).tas
wsTS1=ws_ds.mean(dim=['latitude','longitude'], skipna=True).ws
rhTS1=rh_ds.mean(dim=['latitude','longitude'], skipna=True).rh
wTS1=w700D.mean(dim=['latitude','longitude'], skipna=True).wap
invTS1=inv_ds.mean(dim=['latitude','longitude'], skipna=True).inv
ushearTS1=ushear_ds.mean(dim=['latitude','longitude'], skipna=True).ushear

In [12]:
datetimeindex = t2mTS1.indexes['time'].to_datetimeindex()
t2mTS1['time'] = datetimeindex
wsTS1['time'] = datetimeindex
rhTS1['time'] = datetimeindex
wTS1['time'] = datetimeindex
invTS1['time'] = datetimeindex
ushearTS1['time'] = datetimeindex

<ipython-input-12-cc712bdc98ec>:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  datetimeindex = t2mTS1.indexes['time'].to_datetimeindex()


In [13]:
# shift one day so we can take values of previous day
t2mTS=t2mTS1.shift(time=1)
wsTS=wsTS1.shift(time=1)
rhTS=rhTS1.shift(time=1)
wTS=wTS1.shift(time=1)
invTS=invTS1.shift(time=1)
ushearTS=ushearTS1.shift(time=1)

In [14]:
# Select required time 
t2m=t2mTS[((t2mTS.time.dt.month>11) | (t2mTS.time.dt.month<2)) & (t2mTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
ws=wsTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
inv=invTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
rh=rhTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
w=wTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
ushear=ushearTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

In [15]:
X       = pd.DataFrame([t2m.values,ws.values,rh.values/100.0,inv.values,w.values,ushear.values*100.0,AO5DAll.values,
                        EU5DAll.values],
                       index=['t2m', 'ws', 'rh','inv', 'w', 'ushear', 'AO5D', 'EU5D']).T[:]
X.index = pd.to_datetime(t2m.time.values)
X.describe(include='all')

,t2m,ws,rh,inv,w,ushear,AO5D,EU5D
count,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000
mean,285.931829,2.227997,0.443332,1.988855,0.019420,0.107267,-0.317889,-0.000229
std,2.268915,0.628693,0.173018,1.855361,0.092638,0.081545,0.818169,0.546650
min,278.468842,0.801570,0.132250,-3.688723,-0.394211,-0.251919,-2.832842,-1.545385
25%,284.409401,1.768162,0.300236,0.606693,-0.033737,0.052171,-0.909756,-0.386308
50%,285.939438,2.168082,0.415059,1.933634,0.026514,0.109553,-0.310674,-0.003483
75%,287.431358,2.618625,0.558485,3.431449,0.079204,0.162650,0.293349,0.385351
max,293.514648,4.310729,0.919877,6.666211,0.309867,0.402068,2.659693,1.637002


In [16]:
norm = StandardScaler().fit(X)
    # transform training data
X = norm.transform(X)
X
X = pd.DataFrame(norm.transform(X))
X.index = pd.to_datetime(t2m.time.values)
X.describe(include='all')

,0,1,2,3,4,5,6,7
count,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000
mean,-126.050457,-3.544672,-2.562934,-1.072197,-0.209686,-1.315732,0.388626,0.000419
std,0.440943,1.591335,5.782409,0.539227,10.799713,12.268767,1.222805,1.830168
min,-127.500819,-7.155215,-12.959537,-2.722283,-48.430830,-55.356503,-3.370127,-5.172718
25%,-126.346327,-4.708599,-7.345297,-1.473898,-6.406755,-9.605088,-0.495956,-1.292161
50%,-126.048978,-3.696330,-3.507819,-1.088246,0.617268,-0.971853,0.399409,-0.010475
75%,-125.759037,-2.555924,1.285603,-0.652934,6.759909,7.016763,1.302160,1.291331
max,-124.576806,1.727094,13.363620,0.287191,33.650608,43.038103,4.838808,5.481819


In [17]:
regLin=load('../../March2021/Observation_models/LRModel.joblib') 

In [18]:
yLR=regLin.predict(X)
y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR)}, coords={'time': t2m.time})

In [19]:
dump(y_predLin_ds.yLR,'../Model_plots/INM-CM4-8-LR-Y.joblib')

['../Model_plots/INM-CM4-8-LR-Y.joblib']

In [20]:
regLin2=load('../../March2021/Observation_models/SVRModel.joblib') 
yLR=regLin2.predict(X)
y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR)}, coords={'time': t2m.time})
dump(y_predLin_ds.yLR,'../Model_plots/INM-CM4-8-SVR-Y.joblib')

['../Model_plots/INM-CM4-8-SVR-Y.joblib']